In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Read CSV files into DataFrames
whale_returns = pd.read_csv('whale_returns.csv', index_col='Date')
algo_returns = pd.read_csv('algo_returns.csv', index_col='Date')
sp500_history = pd.read_csv('sp500_history.csv', index_col='Date')

# Convert the 'Date' column to datetime format
whale_returns.index = pd.to_datetime(whale_returns.index, format="%Y-%m-%d")
algo_returns.index = pd.to_datetime(algo_returns.index, format="%Y-%m-%d")
sp500_history.index = pd.to_datetime(sp500_history.index, format="%d-%b-%y") 

# Step 2: Remove null values
whale_returns.dropna(inplace=True)
algo_returns.dropna(inplace=True)
sp500_history.dropna(inplace=True)

# Step 3: Remove non-numeric values and convert data types
whale_returns = whale_returns.apply(pd.to_numeric, errors='coerce')
algo_returns = algo_returns.apply(pd.to_numeric, errors='coerce')
sp500_history['Close'] = sp500_history['Close'].str.replace('$', '').astype(float)

# Step 4: Convert S&P 500 closing prices to daily returns
sp500_returns = sp500_history['Close'].pct_change()

# Step 5: Join all portfolios returns into a single DataFrame
combined_returns = pd.concat([whale_returns, algo_returns, sp500_returns], axis=1)
combined_returns.columns = ['SOROS FUND MANAGEMENT LLC', 'PAULSON & CO.INC.', 'TIGER GLOBAL MANAGEMENT LLC', 'BERKSHIRE HATHAWAY INC', 'Algo 1', 'Algo 2', 'S&P 500']


# Performance Analysis

# Calculate and plot daily returns
combined_returns.plot(figsize=(12, 6))
plt.title("Daily Returns of Portfolios")
plt.xlabel("Date")
plt.ylabel("Returns")
plt.show()

# Calculate and plot cumulative returns
cumulative_returns = (1 + combined_returns).cumprod() - 1
cumulative_returns.plot(figsize=(12, 6))
plt.title("Cumulative Returns of Portfolios")
plt.xlabel("Date")
plt.ylabel("Cumulative Returns")
plt.show()

# Risk Analysis

# Create a box plot for each portfolio's returns
combined_returns.plot.box(figsize=(12, 6))
plt.title("Box Plot of Portfolios Returns")
plt.ylabel("Returns")
plt.show()

# Calculate the standard deviation for each portfolio
std_deviation = combined_returns.std()
print("Standard Deviation:")
print(std_deviation)

# Determine which portfolios are riskier than the S&P 500
riskier_portfolios = std_deviation[std_deviation > std_deviation['S&P 500']].index.tolist()
print("Riskier Portfolios than S&P 500:")
print(riskier_portfolios)

# Calculate the annualized standard deviation
annualized_std_dev = std_deviation * (252 ** 0.5)
print("Annualized Standard Deviation:")
print(annualized_std_dev)

# Rolling Statistics

# Calculate and plot the rolling standard deviation for all portfolios with a 21-day window
rolling_std = combined_returns.rolling(window=21).std()
rolling_std.plot(figsize=(12, 6))
plt.title("Rolling Standard Deviation (21-day Window)")
plt.xlabel("Date")
plt.ylabel("Standard Deviation")
plt.show()

# Calculate and plot the correlation between each stock to determine which portfolios may mimic the S&P 500
correlation = combined_returns.corr()
plt.imshow(correlation, cmap='coolwarm', interpolation='nearest')
plt.colorbar()
plt.title("Correlation between Portfolios")
plt.xticks(range(len(correlation)), correlation.columns, rotation=90)
plt.yticks(range(len(correlation)), correlation.columns)
plt.show()

# Choose one portfolio (e.g., 'Algo 1') and calculate and plot its 60-day rolling beta compared to the S&P 500
portfolio = 'Algo 1'
rolling_covariance = combined_returns[portfolio].rolling(window=60).cov(combined_returns['S&P 500'])
rolling_variance = combined_returns['S&P 500'].rolling(window=60).var()
rolling_beta = rolling_covariance / rolling_variance
rolling_beta.plot(figsize=(12, 6))
plt.title("Rolling Beta - {} vs. S&P 500".format(portfolio))
plt.xlabel("Date")
plt.ylabel("Beta")
plt.show()

# Rolling Statistics Challenge: Exponentially Weighted Average (21-day Half-Life)
ewm_std = combined_returns.ewm(halflife=21).std()
ewm_std.plot(figsize=(12, 6))
plt.title("Exponentially Weighted Moving Average (21-day Half-Life)")
plt.xlabel("Date")
plt.ylabel("Standard Deviation")
plt.show()

# Sharpe Ratios

# Calculate the Sharpe ratios using daily returns
risk_free_rate = 0  # Assuming risk-free rate of return is 0
sharpe_ratios = (combined_returns.mean() - risk_free_rate) / combined_returns.std()

# Visualize the Sharpe ratios using a bar plot
sharpe_ratios.plot(kind='bar', figsize=(12, 6))
plt.title("Sharpe Ratios")
plt.xlabel("Portfolio")
plt.ylabel("Sharpe Ratio")
plt.show()

# Determine whether the algorithmic strategies outperform the market (S&P 500) and the whale portfolios
algo_outperforms_market = sharpe_ratios[['Algo 1', 'Algo 2']].gt(sharpe_ratios['S&P 500']).all()
whale_outperforms_market = sharpe_ratios[['SOROS FUND MANAGEMENT LLC', 'PAULSON & CO.INC.', 'TIGER GLOBAL MANAGEMENT LLC', 'BERKSHIRE HATHAWAY INC']].gt(sharpe_ratios['S&P 500']).all()
print("Algorithmic Strategies Outperform Market (S&P 500):", algo_outperforms_market)
print("Whale Portfolios Outperform Market (S&P 500):", whale_outperforms_market)

# Custom Portfolio

# Download data for chosen stocks (e.g., AAPL, COST, GOOG)
aapl_historical = pd.read_csv('aapl_historical.csv', index_col='Trade DATE', parse_dates=True)
cost_historical = pd.read_csv('cost_historical.csv', index_col='Trade DATE', parse_dates=True)
goog_historical = pd.read_csv('goog_historical.csv', index_col='Trade DATE', parse_dates=True)

# Calculate the portfolio returns assuming equal number of shares per stock
portfolio_returns = (aapl_historical['NOCP'].pct_change() + cost_historical['NOCP'].pct_change() + goog_historical['NOCP'].pct_change()) / 3

# Add the portfolio returns to the DataFrame with the other portfolios
combined_returns['Custom Portfolio'] = portfolio_returns

# Calculate the annualized standard deviation for the custom portfolio
custom_portfolio_std_dev = portfolio_returns.std() * (252 ** 0.5)
print("Annualized Standard Deviation - Custom Portfolio:")
print(custom_portfolio_std_dev)

# Calculate and plot the rolling standard deviation for the custom portfolio with a 21-day window
custom_portfolio_rolling_std = portfolio_returns.rolling(window=21).std()
custom_portfolio_rolling_std.plot(figsize=(12, 6))
plt.title("Rolling Standard Deviation - Custom Portfolio (21-day Window)")
plt.xlabel("Date")
plt.ylabel("Standard Deviation")
plt.show()

# Calculate and plot the correlation between the custom portfolio and the S&P 500
custom_portfolio_corr = portfolio_returns.corr(combined_returns['S&P 500'])
plt.scatter(combined_returns['S&P 500'], portfolio_returns)
plt.title("Correlation - Custom Portfolio vs. S&P 500")
plt.xlabel("S&P 500 Returns")
plt.ylabel("Custom Portfolio Returns")
plt.show()
print("Correlation - Custom Portfolio vs. S&P 500:", custom_portfolio_corr)

# Calculate and plot the beta for the custom portfolio compared to the S&P 500
beta_custom = portfolio_returns.cov(combined_returns['S&P 500']) / combined_returns['S&P 500'].var()
beta_custom.plot(figsize=(12, 6))
plt.title("Beta - Custom Portfolio vs. S&P 500")
plt.xlabel("Date")
plt.ylabel("Beta")
plt.show()

# Calculate the Sharpe ratio for the custom portfolio
sharpe_ratio_custom = (portfolio_returns.mean() - risk_free_rate) / portfolio_returns.std()
print("Sharpe Ratio - Custom Portfolio:", sharpe_ratio_custom)
